In [ ]:
import numpy as np
import pandas as pd
import math

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

#importing tensorflow
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten,BatchNormalization,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import mnist

In [ ]:

from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X = X_train
y = y_train

test_x = X_test

In [ ]:
X = X / 255.0
test_x = test_x / 255.0

In [ ]:
X.shape

(60000, 28, 28)

In [ ]:
# Reshape 
X = X.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)

In [ ]:
# Encode labels 
y = to_categorical(y)

print(f"Label size {y.shape}")

Label size (60000, 10)


In [ ]:
# Split the train and the validation set for the fitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
batch_size = 64

In [ ]:
# With data augmentation to prevent overfitting

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images



train_gen = datagen.flow(X_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
def scheduler(epoch):
   initial_lrate = 0.1
   drop = 0.5
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

In [ ]:
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())   

model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())   

model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())   

model.add(Flatten())
model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
	
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 13, 13, 32)        128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 16)        4624      
_________________________________________________________________
batch_normalization_7 (Batch (None, 11, 11, 16)        64        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 9, 9, 16)          2320      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 16)         

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.001),metrics=["accuracy"])

In [ ]:
filepath="weights.hdf5"
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.datasets import mnist

checkpoint_1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

early_stoping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [ ]:
# Fit the model
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_gen, 
                              epochs = 200,
                              # use_multiprocessing=True, 
                              workers=5,
                              steps_per_epoch = X.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks = [tensorboard_callback,checkpoint_1, early_stoping]
                              
                              )

Epoch 1/200
937/937 [==============================] - 29s 31ms/step - loss: 1.0174 - accuracy: 0.6522 - val_loss: 0.1912 - val_accuracy: 0.9434

Epoch 00001: val_accuracy improved from -inf to 0.94338, saving model to weights.hdf5
Epoch 2/200
937/937 [==============================] - 28s 30ms/step - loss: 0.1804 - accuracy: 0.9451 - val_loss: 0.1750 - val_accuracy: 0.9434

Epoch 00002: val_accuracy did not improve from 0.94338
Epoch 3/200
937/937 [==============================] - 29s 30ms/step - loss: 0.1370 - accuracy: 0.9582 - val_loss: 0.1165 - val_accuracy: 0.9640

Epoch 00003: val_accuracy improved from 0.94338 to 0.96405, saving model to weights.hdf5
Epoch 4/200
937/937 [==============================] - 28s 30ms/step - loss: 0.1116 - accuracy: 0.9643 - val_loss: 0.1002 - val_accuracy: 0.9694

Epoch 00004: val_accuracy improved from 0.96405 to 0.96942, saving model to weights.hdf5
Epoch 5/200
937/937 [==============================] - 28s 30ms/step - loss: 0.0972 - accuracy: 0

# Validation accuracy = 99.14

In [ ]:
0.99143 * 100

99.143